# 5. Topic Modelling Initial Experiments (Supervised Learning)

In [1]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive', force_remount=True) # Adding force_remount=True to force the remounting process

Mounted at /content/drive


In [3]:
import pandas as pd
df_reviews = pd.read_csv('/content/drive/MyDrive/labelled_data_further_processed.csv')
df_reviews

,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49668,49668,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49669,49669,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
49670,49670,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49671,49671,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


In [4]:
! pip install pandas numpy
! pip install gensim nltk
! pip install scikit-learn
! pip install bertopic sentence-transformers umap-learn hdbscan
! pip install matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00


In [5]:
df_reviews_100 = df_reviews.head(100)
df_reviews_100

,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,121,The course content is very good as well as the...,2020-05-05,2,python,2020-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,realization,2 stars,negative
96,96,122,Course material is excellent. I've noticed a h...,2020-04-28,2,python,2020-04,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
97,97,124,"very basic python programming, and the 3rd par...",2018-12-31,2,python,2018-12,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,annoyance,2 stars,negative
98,98,126,"Sorry, the professor has a genial & cheerful p...",2015-09-29,2,python,2015-09,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,gratitude,2 stars,negative


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

LDA Topic Modelling with predefined categories (Supervised)

In [ ]:
# Import necessary libraries
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Text Preprocessing Function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and punctuation
    return tokens

# Apply preprocessing to 'reviews' column
df_reviews_100['processed_reviews'] = df_reviews_100['reviews'].apply(preprocess)

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(df_reviews_100['processed_reviews'])
corpus = [dictionary.doc2bow(review) for review in df_reviews_100['processed_reviews']]

# Train LDA model
num_topics = 14  # Set the number of topics based on your analysis
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Display topics with top words
topics = lda_model.print_topics(num_words=4)
print("Identified Topics and Keywords:")
for idx, topic in topics:
    print(f"Topic {idx+1}: {topic}")

# Function to get dominant topic for each document
def get_dominant_topic(lda_model, corpus):
    topics = []
    for row in lda_model[corpus]:
        row = sorted(row, key=lambda x: x[1], reverse=True)
        topics.append(row[0][0])
    return topics

# Assign dominant topic to each review
df_reviews_100['dominant_topic'] = get_dominant_topic(lda_model, corpus)

# removed 'Scheduling and Timeliness'
topic_mapping = {
    0: 'Assignments',
    1: 'Exams',
    2: 'Lectures',
    3: 'Course Material',
    4: 'Grading and Feedback',
    5: 'Instructor/Professor',
    6: 'Class Participation and Engagement',
    7: 'Online Resources',
    8: 'Course Structure and Organization',
    9: 'Facilities and Equipment',
    10: 'Workload and Difficulty',
    11: 'Learning Outcomes and Skills',
    12: 'Class Environment and Culture',
    13: 'Career Relevance and Application'
}

df_reviews_100['topic_category'] = df_reviews_100['dominant_topic'].map(topic_mapping)

# Display the DataFrame with the new topic labels
print("\nData with Assigned Topic Categories:")
print(df_reviews_100[['reviews', 'dominant_topic', 'topic_category']])


Identified Topics and Keywords:
Topic 1: 0.019*"courses" + 0.019*"free" + 0.010*"course" + 0.010*"better"
Topic 2: 0.029*"still" + 0.019*"upgrade" + 0.019*"submit" + 0.019*"says"
Topic 3: 0.033*"course" + 0.017*"ca" + 0.009*"python" + 0.009*"use"
Topic 4: 0.039*"course" + 0.017*"python" + 0.017*"videos" + 0.017*"little"
Topic 5: 0.016*"course" + 0.015*"like" + 0.015*"video" + 0.014*"python"
Topic 6: 0.038*"course" + 0.021*"programming" + 0.021*"class" + 0.018*"bad"
Topic 7: 0.029*"course" + 0.017*"person" + 0.017*"irrelevant" + 0.014*"lectures"
Topic 8: 0.026*"class" + 0.023*"students" + 0.015*"get" + 0.011*"learning"
Topic 9: 0.022*"program" + 0.012*"programming" + 0.012*"course" + 0.012*"way"
Topic 10: 0.035*"help" + 0.025*"course" + 0.015*"need" + 0.015*"coursera"
Topic 11: 0.038*"course" + 0.024*"lectures" + 0.019*"aid" + 0.019*"financial"
Topic 12: 0.027*"course" + 0.023*"programming" + 0.018*"time" + 0.014*"want"
Topic 13: 0.022*"class" + 0.015*"lectures" + 0.015*"course" + 0.015

In [33]:
df_reviews_100[['reviews', 'dominant_topic', 'topic_category']]

,reviews,dominant_topic,topic_category
0,I did not find this course helpful. The assign...,2,Lectures
1,I got a 95% and maybe I learned something but ...,11,Learning Outcomes and Skills
2,"Two weeks of all sort of nonsense, including m...",11,Learning Outcomes and Skills
3,We tried to use this class to teach a brown ba...,12,Class Environment and Culture
4,"To be honest, I thought the course stumbled an...",4,Grading and Feedback
...,...,...,...
95,The course content is very good as well as the...,13,Career Relevance and Application
96,Course material is excellent. I've noticed a h...,13,Career Relevance and Application
97,"very basic python programming, and the 3rd par...",4,Grading and Feedback
98,"Sorry, the professor has a genial & cheerful p...",7,Online Resources


In [50]:
df_reviews_100.to_csv('/content/drive/MyDrive/topic_modelling_LDA_one.csv')

Adding a list of predefined categories, rather than just one - should combine and compare to results with 1. Important, since, feedback should be as useful as possible.

In [34]:
# Import necessary libraries
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

df_reviews_100_2 = df_reviews.head(100)
# Text Preprocessing Function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and punctuation
    return tokens

# Apply preprocessing to 'reviews' column
df_reviews_100_2['processed_reviews'] = df_reviews_100_2['reviews'].apply(preprocess)

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(df_reviews_100_2['processed_reviews'])
corpus = [dictionary.doc2bow(review) for review in df_reviews_100_2['processed_reviews']]

# Train LDA model
num_topics = 14  # Set the number of topics based on your analysis
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Display topics with top words
topics = lda_model.print_topics(num_words=4)
print("Identified Topics and Keywords:")
for idx, topic in topics:
    print(f"Topic {idx+1}: {topic}")

# Function to get top 3 topics for each document
def get_top_topics(lda_model, corpus, top_n=3):
    top_topics = []
    for row in lda_model[corpus]:
        row = sorted(row, key=lambda x: x[1], reverse=True)  # Sort by probability
        top_n_topics = [topic[0] for topic in row[:top_n]]  # Get top 'n' topic indices
        top_topics.append(top_n_topics)
    return top_topics

# Assign top 3 topics to each review
df_reviews_100_2['top_topics'] = get_top_topics(lda_model, corpus, top_n=3)

# Map topics to categories based on the words in each topic
topic_mapping = {
    0: 'Assignments',
    1: 'Exams',
    2: 'Lectures',
    3: 'Course Material',
    4: 'Grading and Feedback',
    5: 'Instructor/Professor',
    6: 'Class Participation and Engagement',
    7: 'Online Resources',
    8: 'Course Structure and Organization',
    9: 'Facilities and Equipment',
    10: 'Workload and Difficulty',
    11: 'Learning Outcomes and Skills',
    12: 'Class Environment and Culture',
    13: 'Career Relevance and Application'
}

# Function to map topic indices to category names
def map_topics_to_categories(topics, topic_mapping):
    return [topic_mapping[topic] for topic in topics if topic in topic_mapping]

# Apply mapping to top topics for each review
df_reviews_100_2['topic_categories'] = df_reviews_100_2['top_topics'].apply(lambda x: map_topics_to_categories(x, topic_mapping))

# Display the DataFrame with the new topic labels
print("\nData with Assigned Topic Categories:")
print(df_reviews_100_2[['reviews', 'top_topics', 'topic_categories']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Identified Topics and Keywords:
Topic 1: 0.042*"course" + 0.015*"even" + 0.015*"videos" + 0.011*"online"
Topic 2: 0.025*"program" + 0.021*"course" + 0.018*"bad" + 0.011*"got"
Topic 3: 0.018*"looks" + 0.018*"good" + 0.009*"like" + 0.009*"would"
Topic 4: 0.047*"course" + 0.021*"videos" + 0.011*"could" + 0.011*"really"
Topic 5: 0.029*"free" + 0.020*"course" + 0.020*"first" + 0.010*"must"
Topic 6: 0.013*"students" + 0.013*"like" + 0.013*"unprofessional" + 0.013*"completely"
Topic 7: 0.047*"help" + 0.019*"coursera" + 0.019*"people" + 0.018*"course"
Topic 8: 0.029*"lectures" + 0.017*"detail" + 0.012*"taught" + 0.012*"online"
Topic 9: 0.013*"python" + 0.013*"watch" + 0.013*"class" + 0.013*"lunch"
Topic 10: 0.017*"earlier" + 0.009*"professor" + 0.009*"program" + 0.009*"videos"
Topic 11: 0.057*"course" + 0.013*"programming" + 0.013*"even" + 0.013*"would"
Topic 12: 0.032*"name" + 0.032*"certificate" + 0.022*"submit" + 0.022*"says"
Topic 13: 0.022*"python" + 0.022*"learn" + 0.015*"programming" + 

In [35]:
df_reviews_100_2[['reviews', 'top_topics', 'topic_categories']]

,reviews,top_topics,topic_categories
0,I did not find this course helpful. The assign...,[3],[Course Material]
1,I got a 95% and maybe I learned something but ...,"[13, 10]","[Career Relevance and Application, Workload an..."
2,"Two weeks of all sort of nonsense, including m...",[8],[Course Structure and Organization]
3,We tried to use this class to teach a brown ba...,[8],[Course Structure and Organization]
4,"To be honest, I thought the course stumbled an...",[0],[Assignments]
...,...,...,...
95,The course content is very good as well as the...,[10],[Workload and Difficulty]
96,Course material is excellent. I've noticed a h...,[6],[Class Participation and Engagement]
97,"very basic python programming, and the 3rd par...",[13],[Career Relevance and Application]
98,"Sorry, the professor has a genial & cheerful p...",[13],[Career Relevance and Application]


In [51]:
df_reviews_100_2.to_csv('/content/drive/MyDrive/topic_modelling_LDA_multiple.csv')

Using BERTopic for Topic Modelling without predefined catgories.

In [17]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Extract the 'reviews' column from your DataFrame
reviews = df_reviews_100['reviews'].tolist()

# Initialize the BERTopic model
topic_model = BERTopic(embedding_model=SentenceTransformer('paraphrase-MiniLM-L6-v2'), nr_topics=5, verbose=True)

# Fit the BERTopic model to the reviews
topics, probabilities = topic_model.fit_transform(reviews)

# Check if topics are being generated
print("Distinct topics in the model:")
print(set(topics))  # Check the number of distinct topics

# Get the top words for each topic
topics_words = topic_model.get_topic_info()
print("Top words for each topic:")
print(topics_words)

# Visualize the topics interactively
try:
    topic_model.visualize_topics()
except ValueError as e:
    print(f"Error visualizing topics: {e}")

# Visualize the hierarchical structure of the topics
try:
    topic_model.visualize_hierarchy()
except ValueError as e:
    print(f"Error visualizing hierarchy: {e}")

# Visualize the topics in a bar chart format
try:
    topic_model.visualize_barchart()
except ValueError as e:
    print(f"Error visualizing barchart: {e}")

# Add the assigned topics back into the original DataFrame
df_reviews_100['topic'] = topics

# Display the DataFrame with the reviews and their corresponding topics
print("\nDataFrame with topics assigned:")
print(df_reviews_100[['reviews', 'topic']])


2024-11-06 18:23:29,917 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-06 18:23:32,872 - BERTopic - Embedding - Completed ✓
2024-11-06 18:23:32,875 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-06 18:23:33,008 - BERTopic - Dimensionality - Completed ✓
2024-11-06 18:23:33,010 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-06 18:23:33,018 - BERTopic - Cluster - Completed ✓
2024-11-06 18:23:33,019 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-06 18:23:33,036 - BERTopic - Representation - Completed ✓
2024-11-06 18:23:33,037 - BERTopic - Topic reduction - Reducing number of topics
2024-11-06 18:23:33,040 - BERTopic - Topic reduction - Reduced number of topics from 3 to 3


Distinct topics in the model:
{0, 1, -1}
Top words for each topic:
   Topic  Count                      Name  \
0     -1      4   -1_youtube_was_free_not   
1      0     14  0_certificate_my_this_is   
2      1     82       1_the_and_to_course   

                                      Representation  \
0  [youtube, was, free, not, on, to, expecting, t...   
1  [certificate, my, this, is, to, bad, credit, m...   
2  [the, and, to, course, is, for, of, in, it, this]   

                                 Representative_Docs  
0  [last assignment was tricky and was not able t...  
1  [Its very bad for me that it is a non credit c...  
2  [To be honest, I thought the course stumbled a...  
Error visualizing topics: zero-size array to reduction operation maximum which has no identity

DataFrame with topics assigned:
                                              reviews  topic
0   I did not find this course helpful. The assign...      1
1   I got a 95% and maybe I learned something but ...    

In [15]:
df_reviews_100[['reviews','topic']]

,reviews,topic
0,I did not find this course helpful. The assign...,0
1,I got a 95% and maybe I learned something but ...,0
2,"Two weeks of all sort of nonsense, including m...",0
3,We tried to use this class to teach a brown ba...,0
4,"To be honest, I thought the course stumbled an...",0
...,...,...
95,The course content is very good as well as the...,0
96,Course material is excellent. I've noticed a h...,0
97,"very basic python programming, and the 3rd par...",0
98,"Sorry, the professor has a genial & cheerful p...",-1


In [36]:
! pip install openai

In [26]:
import pandas as pd
df_reviews = pd.read_csv('/content/drive/MyDrive/labelled_data_further_processed.csv')
df_reviews

,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
1,1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative
2,2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative
3,3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative
4,4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49668,49668,58724,Great for beginners in programming and people ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49669,49669,58725,nice hope Charles sir gonna come up with more ...,2020-10-08,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,optimism,5 stars,positive
49670,49670,58726,This course is so good. I can easily understan...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
49671,49671,58727,Love it! Even though it isn't my first python ...,2020-10-06,5,python,2020-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,love,5 stars,positive


Testing BART for topic labels

In [6]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import pipeline
import nltk
import os

# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Load the data
df_reviews = df_reviews.head(50)  # Assume this is your original DataFrame with 39,000 reviews

# Define batch size and checkpoint directory
batch_size = 50 # Start with 100 for Colab; adjust if stable
checkpoint_dir = "checkpoints_top_test"
os.makedirs(checkpoint_dir, exist_ok=True)

# Define BART classifier and topics
zero_shot_classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
topics = ["Assignments", "Exams", "Lectures", "Course Material", "Grading", "Instructor", "Workload"]

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# LDA function to get top topics
def get_top_topics(lda_model, corpus, top_n=3):
    top_topics = []
    for row in lda_model[corpus]:
        row = sorted(row, key=lambda x: x[1], reverse=True)
        top_n_topics = [topic[0] for topic in row[:top_n]]
        top_topics.append(top_n_topics)
    return top_topics

# Function to apply BART zero-shot classification
def classify_review_multiple(review, top_n=3):
    result = zero_shot_classifier(review, candidate_labels=topics)
    return result['labels'][:top_n]

# Dictionary for LDA topic mapping
topic_mapping = {
    0: 'Assignments', 1: 'Exams', 2: 'Lectures', 3: 'Course Material',
    4: 'Grading and Feedback', 5: 'Instructor/Professor',
    6: 'Class Participation and Engagement', 7: 'Online Resources',
    8: 'Course Structure and Organization', 9: 'Facilities and Equipment',
    10: 'Workload and Difficulty', 11: 'Learning Outcomes and Skills',
    12: 'Class Environment and Culture', 13: 'Career Relevance and Application'
}

def map_topics_to_categories(topics, topic_mapping):
    return [topic_mapping[topic] for topic in topics if topic in topic_mapping]

# Process the dataset in batches with checkpointing
for i in range(0, len(df_reviews), batch_size):
    batch_num = i // batch_size
    checkpoint_file = os.path.join(checkpoint_dir, f"batch_{batch_num}.csv")

    # Skip processing if checkpoint already exists
    if os.path.exists(checkpoint_file):
        print(f"Skipping batch {batch_num}, checkpoint found.")
        continue

    # Slice the batch
    batch_df = df_reviews.iloc[i:i + batch_size].copy()

    # LDA processing
    batch_df['processed_reviews'] = batch_df['reviews'].apply(preprocess)
    dictionary = corpora.Dictionary(batch_df['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in batch_df['processed_reviews']]
    lda_model = LdaModel(corpus, num_topics=14, id2word=dictionary, passes=15)
    batch_df['LDA_Topic'] = get_top_topics(lda_model, corpus, top_n=3)
    batch_df['LDA_Topic'] = batch_df['LDA_Topic'].apply(lambda x: map_topics_to_categories(x, topic_mapping))

    # BART processing
    batch_df['BART_Topic'] = batch_df['reviews'].apply(classify_review_multiple)

    # Save the batch to a checkpoint file
    batch_df.to_csv(checkpoint_file, index=False)
    print(f"Processed and saved batch {batch_num}.")

# Load all checkpoints and concatenate them
all_batches = []
for file in sorted(os.listdir(checkpoint_dir)):
    if file.startswith("batch_") and file.endswith(".csv"):
        batch_df = pd.read_csv(os.path.join(checkpoint_dir, file))
        all_batches.append(batch_df)

# Concatenate all processed batches into a single DataFrame
processed_df = pd.concat(all_batches, ignore_index=True)

# Ensure that the final DataFrame includes all original columns plus the new topic labels
# This should match the structure and order of the original DataFrame
final_df = pd.concat([df_reviews.reset_index(drop=True), processed_df[['LDA_Topic', 'BART_Topic']]], axis=1)

# Display final DataFrame with all original columns plus new topic labels
print(final_df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processed and saved batch 0.
   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                             reviews date_reviews  rating  \
0  I did not find this course helpful. The assign...   2017-10-05       1   
1  I got a 95% and maybe I learned something but ...   2017-08-06       1   
2  Two weeks of all sort of nonsense, including m...   2019-03-28       1   
3  We tried to use this class to teach a brown ba...   2015-10-20       1   
4  To be honest, I thought the course stumbled an...   2016-05-11       1   

  course_id    month       departments  \
0    python  2017-10  Computer Science   
1    python  2017-08  Computer Science   
2    python  2019-03  Computer Science   
3    python  2015-10  Computer Science   
4    python  2016-05  Computer Science   

                                                name             ins

In [7]:
final_df

,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review,LDA_Topic,BART_Topic
0,0,0,I did not find this course helpful. The assign...,2017-10-05,1,python,2017-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative,['Class Participation and Engagement'],"['Assignments', 'Course Material', 'Instructor']"
1,1,1,I got a 95% and maybe I learned something but ...,2017-08-06,1,python,2017-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative,['Assignments'],"['Course Material', 'Instructor', 'Lectures']"
2,2,2,"Two weeks of all sort of nonsense, including m...",2019-03-28,1,python,2019-03,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,surprise,1 star,negative,['Class Environment and Culture'],"['Workload', 'Instructor', 'Course Material']"
3,3,3,We tried to use this class to teach a brown ba...,2015-10-20,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,admiration,2 stars,negative,['Facilities and Equipment'],"['Course Material', 'Instructor', 'Workload']"
4,4,4,"To be honest, I thought the course stumbled an...",2016-05-11,1,python,2016-05,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative,"['Career Relevance and Application', 'Assignme...","['Course Material', 'Instructor', 'Lectures']"
5,5,5,"Most vague info, assignments don't even have ...",2019-01-13,1,python,2019-01,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,1 star,negative,['Lectures'],"['Assignments', 'Course Material', 'Instructor']"
6,6,6,I am very disappointed with this programming c...,2015-10-08,1,python,2015-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disappointment,2 stars,negative,['Assignments'],"['Instructor', 'Course Material', 'Assignments']"
7,7,7,This course has been dumbed down so much that ...,2015-12-20,1,python,2015-12,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,neutral,2 stars,negative,['Facilities and Equipment'],"['Course Material', 'Instructor', 'Lectures']"
8,8,8,This Course is bad! I try to ask for help and ...,2016-10-21,1,python,2016-10,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,approval,1 star,negative,"['Instructor/Professor', 'Facilities and Equip...","['Course Material', 'Instructor', 'Lectures']"
9,9,9,If I were to compare the quality of this clas...,2020-08-19,1,python,2020-08,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,confusion,2 stars,negative,"['Class Participation and Engagement', 'Facili...","['Assignments', 'Instructor', 'Course Material']"
